# **Name: Varun Magotra**
# **Roll Number: 14**
# **Batch: A**

# Lab 7: Text Analysis

In [ ]:
import os
import urllib.request
import matplotlib.pyplot as plt
from scipy import spatial
import numpy as np
import pandas as pd
from collections import Counter
from math import sqrt
from itertools import combinations 
import torch
import torchtext
from nltk.stem import WordNetLemmatizer
import nltk
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Problem 1.1 – Loading the Dataset

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
stemmer = SnowballStemmer("english")

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/emails.csv")

In [ ]:
data.describe()

,spam
count,5728.000000
mean,0.238827
std,0.426404
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
data

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
...,...,...
5723,Subject: re : research and development charges...,0
5724,"Subject: re : receipts from visit jim , than...",0
5725,Subject: re : enron case study update wow ! a...,0
5726,"Subject: re : interest david , please , call...",0


## How many of the emails are spam?

In [ ]:
data[data['spam']==0]

,text,spam
1368,"Subject: hello guys , i ' m "" bugging you "" f...",0
1369,Subject: sacramento weather station fyi - - ...,0
1370,Subject: from the enron india newsdesk - jan 1...,0
1371,Subject: re : powerisk 2001 - your invitation ...,0
1372,Subject: re : resco database and customer capt...,0
...,...,...
5723,Subject: re : research and development charges...,0
5724,"Subject: re : receipts from visit jim , than...",0
5725,Subject: re : enron case study update wow ! a...,0
5726,"Subject: re : interest david , please , call...",0


In [ ]:
data[data['spam']==1]

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
...,...,...
1363,Subject: are you ready to get it ? hello ! v...,1
1364,Subject: would you like a $ 250 gas card ? do...,1
1365,"Subject: immediate reply needed dear sir , i...",1
1366,Subject: wanna see me get fisted ? fist bang...,1


# Data Cleaning

In [ ]:
data['text'] = data['text'].str.replace('Subject:', '')

In [ ]:
pattern="[^a-zA-Z0-9]"
data['text']=[re.sub(pattern," ",text.lower()) for text in data['text']]
# data['text']=[re.sub(r'h'," ",text) for text in data['text']]


In [ ]:
# punc ='''!()-[]{};:'"\,<>./?@#$%^&*_~'''
# data['text']=[re.sub(punc," ",text.lower()) for text in data['text']]
import string
data['text']=data['text'].str.replace('[{}]'.format(string.punctuation), '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data['text']=[nltk.word_tokenize(text) for text in data['text']]

In [ ]:
data['text'] = data['text'].apply(lambda x: [stemmer.stem(y) for y in x]) 

In [ ]:
data

,text,spam
0,"[natur, irresist, your, corpor, ident, lt, is,...",1
1,"[the, stock, trade, gunsling, fanni, is, merri...",1
2,"[unbeliev, new, home, made, easi, im, want, to...",1
3,"[4, color, print, special, request, addit, inf...",1
4,"[do, not, have, money, get, softwar, cds, from...",1
...,...,...
5723,"[re, research, and, develop, charg, to, gpg, h...",0
5724,"[re, receipt, from, visit, jim, thank, again, ...",0
5725,"[re, enron, case, studi, updat, wow, all, on, ...",0
5726,"[re, interest, david, pleas, call, shirley, cr...",0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer(max_features=28000)
x=vectorizer.fit_transform(" ".join(text) for text in data['text']).toarray()

In [ ]:
x.shape

(5728, 28000)

In [ ]:
dtm=pd.DataFrame(x, columns=vectorizer.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
dtm.head()

,00,000,0000,000000,00000000,0000000000,000000000003619,000000000003991,000000000003997,000000000005168,...,zwzm,zxghlajf,zyban,zyc,zygoma,zymg,zzmacmac,zzn,zzncacst,zzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Building machine learning models

In [ ]:
from sklearn.model_selection import train_test_split

train, test, y_train, y_test = train_test_split(dtm,data['spam'],test_size=0.2)

In [ ]:
from sklearn import tree
model=tree.DecisionTreeClassifier()
classifier = model.fit(train, y_train)

In [ ]:
classifier.score(test,y_test)

0.9537521815008726

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model=RandomForestClassifier()
random_classifier=model.fit(train, y_train)

In [ ]:
random_classifier.score(test,y_test)

0.9712041884816754

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score
predCart = classifier.predict(test)
predRF = random_classifier.predict(test)
print(f"Cart AUC: {roc_auc_score(y_test, predCart)}")
print(f"RF AUC: {roc_auc_score(y_test, predRF)}")

Cart AUC: 0.9531246754433703
RF AUC: 0.9927797833935018


# Inference

The email dataset had uneven class distribution with 1368 spam emails and 4360 ham emails. In this experiment, I performed text analysis for separating spam email from ham. A spam classifier can potentially benefit from including the frequency of the word that appears in every email. Random Forest model outperformed CART model in terms of Accuracy and AUC. This is because Random forests consist of multiple single trees each based on a random sample of the training data. They are typically more accurate than single decision trees.

The DTM is a sparse matrix as there are many 0s and rare values.
While a single decision tree like CART is often pruned, a random forest tree is fully grown and unpruned.